# Predictive Analysis on Clinical Trial Outcomes

#### Investigator
- [Nicholas S. McBride, PhD](https://www.linkedin.com/in/nsmcbride/)

## Section 2: Data Cleaning

##### [ClinicalTrials.gov](https://clinicaltrials.gov/) Reference Links

- [API documentation](https://clinicaltrials.gov/data-api/api)
- [Data structure](https://clinicaltrials.gov/data-api/about-api/study-data-structure)
- [ESSIE query syntax](https://clinicaltrials.gov/find-studies/constructing-complex-search-queries)

In [1]:
# Imports
import numpy as np
import pandas as pd
import bz2
import pickle
from os import listdir
from time import time

In [2]:
# Google Colab environment
# Determine if runtime is a Google Colab enviroment
colab = 'google.colab' in str(get_ipython())

# Workspace settings
if colab:
    project_path = data_path = './'
else:
    project_path = '../'
    data_path = project_path + 'data/'

### Load Data

In [3]:
# Load list of in scope fields
with open(project_path + 'inscope_fields.txt') as file:
    fields = file.read().splitlines()
print(f'Fields selected: {len(fields)}')

# Load API field information from pickle
field_info = pd.read_pickle(data_path + 'field_info.pkl.bz2')

# Create dicts to lookup field name from piece name and the  reverse
field_to_piece = dict(field_info[['field', 'piece']].values)
piece_to_field = dict(field_info[['piece', 'field']].values)

Fields selected: 115


In [4]:
# Find data files
file_list = listdir(data_path)

# Filter the file list
file_list = [file for file in file_list if file[0] != "." and 'json' in file]

# Use the most recent download
file_list.sort(reverse=True)
print(f'ClinicalTrials.gov API download files found: {file_list}')
filename = data_path + file_list[0]

# Load the saved API data
print(f'Loading file {filename}...')
start_time = time()
with bz2.BZ2File(filename, 'r') as file:
    studies_json = pickle.load(file)
print(f'Data loaded in {round(time() - start_time, 1)}s')

ClinicalTrials.gov API download files found: ['2024-05-22T142524_studies_json.pkl.bz2', '2024-05-18T204549_studies_json.pkl.bz2']
Loading file ../data/2024-05-22T142524_studies_json.pkl.bz2...
Data loaded in 24.5s


In [5]:
# Convert to a pd.DateFrame
studies = pd.json_normalize(studies_json)

# Unload the JSON from memory
del studies_json

studies.head()

,protocolSection.identificationModule.nctId,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess,protocolSection.statusModule.startDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.type,protocolSection.statusModule.completionDateStruct.date,...,protocolSection.identificationModule.orgStudyIdInfo.type,protocolSection.referencesModule.availIpds,protocolSection.statusModule.expandedAccessInfo.statusForNctId,protocolSection.oversightModule.isUnapprovedDevice,protocolSection.designModule.expandedAccessTypes.intermediate,protocolSection.designModule.expandedAccessTypes.treatment,documentSection.largeDocumentModule.noSap,protocolSection.designModule.expandedAccessTypes.individual,protocolSection.oversightModule.fdaaa801Violation,protocolSection.oversightModule.isPpsd
0,NCT01871896,"University of California, San Francisco",OTHER,2023-07,COMPLETED,False,2014-01,2022-06,ACTUAL,2022-06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NCT00330096,Société des Produits Nestlé (SPN),INDUSTRY,2013-10,COMPLETED,False,2006-03,NaN,NaN,2009-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NCT02247596,Khoo Teck Puat Hospital,OTHER,2014-09,COMPLETED,False,2009-07,2012-10,ACTUAL,2012-10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NCT03110796,Laboratoires URGO,INDUSTRY,2019-01,UNKNOWN,False,2017-07-31,2020-01,ESTIMATED,2020-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NCT00405496,"Sirion Therapeutics, Inc.",INDUSTRY,2006-11,COMPLETED,False,2000-03,NaN,NaN,2001-04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Cleaning

#### Unpack Lists of Dictionaries

In [6]:
# Functions for data cleaning

def find_list_columns(df):
    """
    Iterates over the columns of a pd.DataFrame to determine is any of the values are a list type.
    Returns a list of the names of columns that contain lists.
    """
    return list(df.columns[df.map(lambda value: isinstance(value, list)).any()])

def unpack_dict_list(series, field_to_piece=field_to_piece):
    """
    Unpack lists of dictionary obects into seperate columns for each piece
    """

    df = pd.DataFrame(index=series.index)
    
    for row, dict_list in enumerate(series):
        row_index = series.index[row]

        # Check for NaN
        if dict_list != dict_list:
            continue

        # Check if it's not a list of dictionaries, return the original object if it is not
        if not isinstance(dict_list[0], dict):

            # Create the column with all NoneType values if it doesn't exist
            if series.name not in df.columns:
                df[series.name] = None
            
            df.at[row_index, series.name] = dict_list
            continue
        
        for dictionary in dict_list:
            for key, value in dictionary.items():

                # Check for double-nested dict
                if isinstance(value, dict):

                    # Create the column with NoneType values if it doesn't exist
                    if key not in df.columns:
                        df[key] = None
                    
                    df.at[row_index, key] = value
                    continue
                
                piece = field_to_piece[series.name + '.' + key]
                
                if piece in df.columns:

                    df_element = df.at[row_index, piece]

                    # Check if there is a NaN in there, replace if so
                    if df_element != df_element or df_element is None:
                        df.at[row_index, piece] = [value]
                        
                    elif isinstance(df_element, list):
                        df.at[row_index, piece].append(value)
                    else: # This should not run
                        print('Found element that is not a list.')
                        df.at[row_index, piece] = [df_element, value]
                        
                else:
                    # Create the column with all None
                    df[piece] = None

                    # Set the value in the specified element
                    df.at[row_index, piece] = [value]
    return df

In [7]:
# Determine which columns contain embedded lists to be unpacked
list_cols = find_list_columns(studies)
list_cols

['protocolSection.conditionsModule.conditions',
 'protocolSection.conditionsModule.keywords',
 'protocolSection.designModule.phases',
 'protocolSection.armsInterventionsModule.armGroups',
 'protocolSection.armsInterventionsModule.interventions',
 'protocolSection.outcomesModule.primaryOutcomes',
 'protocolSection.outcomesModule.secondaryOutcomes',
 'protocolSection.eligibilityModule.stdAges',
 'protocolSection.contactsLocationsModule.overallOfficials',
 'protocolSection.contactsLocationsModule.locations',
 'protocolSection.referencesModule.references',
 'protocolSection.sponsorCollaboratorsModule.collaborators',
 'protocolSection.designModule.designInfo.maskingInfo.whoMasked',
 'protocolSection.contactsLocationsModule.centralContacts',
 'protocolSection.identificationModule.secondaryIdInfos',
 'protocolSection.ipdSharingStatementModule.infoTypes',
 'protocolSection.outcomesModule.otherOutcomes',
 'protocolSection.referencesModule.availIpds']

In [8]:
# Unpack the dictionaries embedded in lists
for col in list_cols:
    print(f'Unpacking dictionary lists in {col}...')
    studies = pd.merge(studies.drop(columns=col), unpack_dict_list(studies[col], field_to_piece), left_index=True, right_index=True)

studies.head()

Unpacking dictionary lists in protocolSection.conditionsModule.conditions...
Unpacking dictionary lists in protocolSection.conditionsModule.keywords...
Unpacking dictionary lists in protocolSection.designModule.phases...
Unpacking dictionary lists in protocolSection.armsInterventionsModule.armGroups...
Unpacking dictionary lists in protocolSection.armsInterventionsModule.interventions...
Unpacking dictionary lists in protocolSection.outcomesModule.primaryOutcomes...
Unpacking dictionary lists in protocolSection.outcomesModule.secondaryOutcomes...
Unpacking dictionary lists in protocolSection.eligibilityModule.stdAges...
Unpacking dictionary lists in protocolSection.contactsLocationsModule.overallOfficials...
Unpacking dictionary lists in protocolSection.contactsLocationsModule.locations...
Unpacking dictionary lists in protocolSection.referencesModule.references...
Unpacking dictionary lists in protocolSection.sponsorCollaboratorsModule.collaborators...
Unpacking dictionary lists in pr

,protocolSection.identificationModule.nctId,protocolSection.identificationModule.organization.fullName,protocolSection.identificationModule.organization.class,protocolSection.statusModule.statusVerifiedDate,protocolSection.statusModule.overallStatus,protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess,protocolSection.statusModule.startDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.date,protocolSection.statusModule.primaryCompletionDateStruct.type,protocolSection.statusModule.completionDateStruct.date,...,CollaboratorName,CollaboratorClass,protocolSection.designModule.designInfo.maskingInfo.whoMasked,CentralContactName,CentralContactRole,SecondaryIdType,protocolSection.ipdSharingStatementModule.infoTypes,OtherOutcomeMeasure,OtherOutcomeTimeFrame,AvailIPDType
0,NCT01871896,"University of California, San Francisco",OTHER,2023-07,COMPLETED,False,2014-01,2022-06,ACTUAL,2022-06,...,None,None,None,None,None,None,None,None,None,None
1,NCT00330096,Société des Produits Nestlé (SPN),INDUSTRY,2013-10,COMPLETED,False,2006-03,NaN,NaN,2009-04,...,[Institut National de Recherche pour l'Agricul...,"[OTHER, UNKNOWN]",None,None,None,None,None,None,None,None
2,NCT02247596,Khoo Teck Puat Hospital,OTHER,2014-09,COMPLETED,False,2009-07,2012-10,ACTUAL,2012-10,...,"[National Medical Research Council (NMRC), Sin...",[OTHER_GOV],"[PARTICIPANT, INVESTIGATOR]",None,None,None,None,None,None,None
3,NCT03110796,Laboratoires URGO,INDUSTRY,2019-01,UNKNOWN,False,2017-07-31,2020-01,ESTIMATED,2020-01,...,None,None,"[INVESTIGATOR, OUTCOMES_ASSESSOR]","[Anne Sauvadet, PhD, Olivier Tacca, PhD]","[CONTACT, CONTACT]",None,None,None,None,None
4,NCT00405496,"Sirion Therapeutics, Inc.",INDUSTRY,2006-11,COMPLETED,False,2000-03,NaN,NaN,2001-04,...,None,None,None,None,None,None,None,None,None,None


#### Rename Columns

Rename the columns using the unique piece name from documentation to improve readability and typability.

In [9]:
# Rename columns using piece names for better readability
studies.rename(columns=field_to_piece, inplace=True)
studies.columns

Index(['NCTId', 'OrgFullName', 'OrgClass', 'StatusVerifiedDate',
       'OverallStatus', 'HasExpandedAccess', 'StartDate',
       'PrimaryCompletionDate', 'PrimaryCompletionDateType', 'CompletionDate',
       'CompletionDateType', 'StudyFirstSubmitDate', 'StudyFirstSubmitQCDate',
       'LastUpdateSubmitDate', 'ResponsiblePartyType', 'LeadSponsorName',
       'LeadSponsorClass', 'OversightHasDMC', 'StudyType', 'DesignAllocation',
       'DesignInterventionModel', 'DesignPrimaryPurpose', 'DesignMasking',
       'EnrollmentCount', 'EnrollmentType', 'HealthyVolunteers', 'Sex',
       'MinimumAge', 'MaximumAge', 'ResponsiblePartyInvestigatorTitle',
       'ResponsiblePartyInvestigatorAffiliation', 'StartDateType',
       'IsFDARegulatedDrug', 'IsFDARegulatedDevice', 'IPDSharing',
       'DispFirstSubmitDate', 'DispFirstSubmitQCDate', 'WhyStopped',
       'PatientRegistry', 'TargetDuration', 'DesignObservationalModel',
       'DesignTimePerspective', 'SamplingMethod', 'BioSpecRetention',
  

In [10]:
# Redetermine which columns contain embedded lists after unpacking dictionaries
list_cols = find_list_columns(studies)
list_cols

['Condition',
 'Keyword',
 'Phase',
 'ArmGroupLabel',
 'ArmGroupType',
 'ArmGroupInterventionName',
 'InterventionType',
 'InterventionName',
 'InterventionArmGroupLabel',
 'PrimaryOutcomeMeasure',
 'PrimaryOutcomeTimeFrame',
 'SecondaryOutcomeMeasure',
 'SecondaryOutcomeTimeFrame',
 'StdAge',
 'OverallOfficialName',
 'OverallOfficialAffiliation',
 'OverallOfficialRole',
 'LocationState',
 'LocationCountry',
 'LocationStatus',
 'ReferenceType',
 'CollaboratorName',
 'CollaboratorClass',
 'DesignWhoMasked',
 'CentralContactName',
 'CentralContactRole',
 'SecondaryIdType',
 'IPDSharingInfoType',
 'OtherOutcomeMeasure',
 'OtherOutcomeTimeFrame',
 'AvailIPDType']

#### Convert Data Types

Take a look at columns that are listed as boolean data types to make sure there are no nested fields or all one value.

In [11]:
# Investigate 'boolean' fields
bool_pieces = field_info.loc[(field_info['type'] == 'BOOLEAN') & (field_info['field'].isin(fields)), 'piece']
studies[studies.columns[studies.columns.isin(bool_pieces)]].describe().T

,count,unique,top,freq
HasExpandedAccess,378669,2,False,378058
OversightHasDMC,311704,2,False,202307
HealthyVolunteers,374752,2,False,269016
IsFDARegulatedDrug,166824,2,False,140939
IsFDARegulatedDevice,166797,2,False,157644
PatientRegistry,60846,2,False,53749
IsUSExport,23821,2,False,16658
GenderBased,8720,1,True,8720
IsUnapprovedDevice,2083,1,True,2083
ExpAccTypeIntermediate,177,1,True,177


Each of the columns contains either True/False/NaN or True/NaN without any False values. None are uniformly filled with the same value. We will have to be careful in converting the data types as `np.nan` values are converted to `True` boolean values by default. We will convert them to a `category`.

In [12]:
bool(np.nan)

True

In [13]:
# Assign correct data types based on API documentation
field_info['type'].unique()

array(['ENUM', 'STRING', 'DATE', 'INTEGER', 'NUMBER', 'BOOLEAN'],
      dtype=object)

In [14]:
# Function for data type parsing
def parse_age(age_string):
    """
    Function to parse the age values given by the ClinicalTrials.gov API.
    These values are given in the formate of a 'number time-unit'
    """
    if isinstance(age_string, float):
        return age_string
        
    age = age_string.split()
    
    if age[1].lower().startswith('year'):
        return np.float32(age[0])
    elif age[1].lower().startswith('month'):
        return np.float32(float(age[0]) / 12)
    elif age[1].lower().startswith('week'):
        return np.float32(float(age[0]) / 52.1429)
    elif age[1].lower().startswith('day'):
        return np.float32(float(age[0]) / 365.25)
    elif age[1].lower().startswith('hour'):
        return np.float32(float(age[0]) / 8766)
    elif age[1].lower().startswith('minute'):
        return np.float32(float(age[0]) / 525960)
        
    else:
        raise Exception(f'Unexpected unit {age[1]}')

In [15]:
studies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383135 entries, 0 to 383134
Data columns (total 86 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   NCTId                                    383135 non-null  object 
 1   OrgFullName                              383135 non-null  object 
 2   OrgClass                                 383135 non-null  object 
 3   StatusVerifiedDate                       383135 non-null  object 
 4   OverallStatus                            383135 non-null  object 
 5   HasExpandedAccess                        378669 non-null  object 
 6   StartDate                                378873 non-null  object 
 7   PrimaryCompletionDate                    362437 non-null  object 
 8   PrimaryCompletionDateType                362374 non-null  object 
 9   CompletionDate                           367623 non-null  object 
 10  CompletionDateType              

In [16]:
# Convert data types
for col in studies.drop(columns=list_cols).columns:
        
    doc_dtype = field_info.loc[field_info["piece"] == col, "type"].values[0]
    df_dtype = studies[col].dtype

    # Handle incorrectly labeled types
    if col in ['MinimumAge', 'MaximumAge', 'TargetDuration']:
        print(f'Converting {col} ({df_dtype}) documented as {doc_dtype} to float32 in years')
        studies[col] = studies[col].map(parse_age).astype(np.float32)
        
    elif doc_dtype == 'DATE':
        print(f'Converting {col} ({df_dtype}) documented as {doc_dtype} to datetime')
        studies[col] = pd.to_datetime(studies[col], format='ISO8601')
        
    elif doc_dtype in ['BOOLEAN', 'ENUM']:
        print(f'Converting {col} ({df_dtype}) documented as {doc_dtype} to category')
        studies[col] = studies[col].astype('category')
        
    elif doc_dtype in ['INTEGER', 'NUMBER']:
            print(f'Converting {col} ({df_dtype}) documented as {doc_dtype} to float32 to preserve NA values')
            studies[col] = studies[col].astype(np.float32)

Converting OrgClass (object) documented as ENUM to category
Converting StatusVerifiedDate (object) documented as DATE to datetime
Converting OverallStatus (object) documented as ENUM to category
Converting HasExpandedAccess (object) documented as BOOLEAN to category
Converting StartDate (object) documented as DATE to datetime
Converting PrimaryCompletionDate (object) documented as DATE to datetime
Converting PrimaryCompletionDateType (object) documented as ENUM to category
Converting CompletionDate (object) documented as DATE to datetime
Converting CompletionDateType (object) documented as ENUM to category
Converting StudyFirstSubmitDate (object) documented as DATE to datetime
Converting StudyFirstSubmitQCDate (object) documented as DATE to datetime
Converting LastUpdateSubmitDate (object) documented as DATE to datetime
Converting ResponsiblePartyType (object) documented as ENUM to category
Converting LeadSponsorClass (object) documented as ENUM to category
Converting OversightHasDMC (

In [17]:
studies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383135 entries, 0 to 383134
Data columns (total 86 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   NCTId                                    383135 non-null  object        
 1   OrgFullName                              383135 non-null  object        
 2   OrgClass                                 383135 non-null  category      
 3   StatusVerifiedDate                       383135 non-null  datetime64[ns]
 4   OverallStatus                            383135 non-null  category      
 5   HasExpandedAccess                        378669 non-null  category      
 6   StartDate                                378873 non-null  datetime64[ns]
 7   PrimaryCompletionDate                    362437 non-null  datetime64[ns]
 8   PrimaryCompletionDateType                362374 non-null  category      
 9   CompletionDate            

### Save DataFrame to File

In [18]:
# Pickle the DataFrame
studies_df_filename = data_path + 'studies_df.pkl.bz2'
studies.to_pickle(studies_df_filename, compression='bz2')